In [1]:
import sys
sys.path.insert(0, '../..')
from allel.io_vcf_read import read_vcf, read_vcf_chunks, vcf_to_npz, vcf_to_hdf5, vcf_to_zarr
from allel.opt.io_vcf_read import (iter_vcf, 
#                                    CalldataParser_parse, 
#                                    GenotypeInt8Parser_parse, 
#                                    ParserContext_next, 
#                                    BufferedReader_read
                                  )

In [2]:
prof_vcf_fn = '/localssd/kwiat/vector/ag1000g/release/phase1.AR3.1/haplotypes/specific_regions/PARA/2L_2358158_2431617.vcf.gz'
sample_vcf_fn = '../../fixture/sample.vcf'

In [3]:
!cat {sample_vcf_fn}

##fileformat=VCFv4.0
##fileDate=20090805
##source=myImputationProgramV3.1
##reference=1000GenomesPilot-NCBI36
##phasing=partial
##INFO=<ID=NS,Number=1,Type=Integer,Description="Number of Samples With Data">
##INFO=<ID=AN,Number=1,Type=Integer,Description="Total number of alleles in called genotypes">
##INFO=<ID=AC,Number=.,Type=Integer,Description="Allele count in genotypes, for each ALT allele, in the same order as listed">
##INFO=<ID=DP,Number=1,Type=Integer,Description="Total Depth">
##INFO=<ID=AF,Number=.,Type=Float,Description="Allele Frequency">
##INFO=<ID=AA,Number=1,Type=String,Description="Ancestral Allele">
##INFO=<ID=DB,Number=0,Type=Flag,Description="dbSNP membership, build 129">
##INFO=<ID=H2,Number=0,Type=Flag,Description="HapMap2 membership">
##FILTER=<ID=s50,Description="Less than 50% of samples have data">
##FILTER=<ID=q10,Description="Quality below 10">
##FORMAT=<ID=GT,Number=1,Type=String,Description="Genotype">
##FORMAT=<ID=GQ,Number=1,Type=Integer,Description="Geno

In [4]:
headers, chunks = read_vcf_chunks(sample_vcf_fn, buffer_size=2**15, chunk_length=3, 
                                  fields='*',
                                  types={'CHROM': 'S4', 'ID': 'S20', 'DP': 'i8', 'AF': 'f8', 'HQ': 'i1', 'GQ': 'i1'},
                                  numbers={'ALT': 2, 'AF': 2, 'AA': 2}
                                 )
headers.samples

['NA00001', 'NA00002', 'NA00003']

In [5]:
headers.filters

{'q10': {'Description': 'Quality below 10', 'ID': 'q10'},
 's50': {'Description': 'Less than 50% of samples have data', 'ID': 's50'}}

In [6]:
headers.infos

{'AA': {'Description': 'Ancestral Allele',
  'ID': 'AA',
  'Number': '1',
  'Type': 'String'},
 'AC': {'Description': 'Allele count in genotypes, for each ALT allele, in the same order as listed',
  'ID': 'AC',
  'Number': '.',
  'Type': 'Integer'},
 'AF': {'Description': 'Allele Frequency',
  'ID': 'AF',
  'Number': '.',
  'Type': 'Float'},
 'AN': {'Description': 'Total number of alleles in called genotypes',
  'ID': 'AN',
  'Number': '1',
  'Type': 'Integer'},
 'DB': {'Description': 'dbSNP membership, build 129',
  'ID': 'DB',
  'Number': '0',
  'Type': 'Flag'},
 'DP': {'Description': 'Total Depth',
  'ID': 'DP',
  'Number': '1',
  'Type': 'Integer'},
 'H2': {'Description': 'HapMap2 membership',
  'ID': 'H2',
  'Number': '0',
  'Type': 'Flag'},
 'NS': {'Description': 'Number of Samples With Data',
  'ID': 'NS',
  'Number': '1',
  'Type': 'Integer'}}

In [7]:
headers.formats

{'DP': {'Description': 'Read Depth',
  'ID': 'DP',
  'Number': '1',
  'Type': 'Integer'},
 'GQ': {'Description': 'Genotype Quality',
  'ID': 'GQ',
  'Number': '1',
  'Type': 'Integer'},
 'GT': {'Description': 'Genotype',
  'ID': 'GT',
  'Number': '1',
  'Type': 'String'},
 'HQ': {'Description': 'Haplotype Quality',
  'ID': 'HQ',
  'Number': '2',
  'Type': 'Integer'}}

In [8]:
chunks = list(chunks)
len(chunks)

3

In [9]:
sorted(chunks[0].keys())

['calldata/DP',
 'calldata/GQ',
 'calldata/GT',
 'calldata/HQ',
 'variants/AA',
 'variants/AC',
 'variants/AF',
 'variants/ALT',
 'variants/AN',
 'variants/CHROM',
 'variants/DB',
 'variants/DP',
 'variants/FILTER_PASS',
 'variants/FILTER_q10',
 'variants/FILTER_s50',
 'variants/H2',
 'variants/ID',
 'variants/NS',
 'variants/POS',
 'variants/QUAL',
 'variants/REF']

In [10]:
sum([r['variants/CHROM'].shape[0] for r in chunks])

9

In [11]:
chunks[0]['variants/CHROM']

array([b'19', b'19', b'20'], 
      dtype='|S4')

In [12]:
chunks[-1]['variants/CHROM']

array([b'20', b'20', b'X'], 
      dtype='|S4')

In [13]:
chunks[0]['variants/POS']

array([  111,   112, 14370], dtype=int32)

In [14]:
chunks[-1]['variants/POS']

array([1234567, 1235237,      10], dtype=int32)

In [15]:
chunks[0]['variants/ID']

array([b'.', b'.', b'rs6054257'], 
      dtype='|S20')

In [16]:
chunks[0]['variants/REF']

array([b'A', b'A', b'G'], 
      dtype='|S1')

In [17]:
chunks[0]['variants/ALT']

array([[b'C', b''],
       [b'G', b''],
       [b'A', b'']], 
      dtype='|S1')

In [18]:
chunks[-1]['variants/ALT']

array([[b'G', b'G'],
       [b'.', b''],
       [b'A', b'A']], 
      dtype='|S1')

In [19]:
chunks[0]['variants/QUAL']

array([  9.60000038,  10.        ,  29.        ], dtype=float32)

In [20]:
chunks[0]['variants/FILTER_PASS']

array([False, False,  True], dtype=bool)

In [21]:
chunks[0]['variants/NS']

array([-1, -1,  3], dtype=int32)

In [22]:
chunks[0]['variants/DP']

array([-1, -1, 14])

In [23]:
chunks[0]['variants/AF']

array([[ nan,  nan],
       [ nan,  nan],
       [ 0.5,  nan]])

In [24]:
chunks[-1]['variants/AC']

array([[ 3,  1, -1],
       [-1, -1, -1],
       [-1, -1, -1]], dtype=int32)

In [25]:
chunks[0]['variants/NS']

array([-1, -1,  3], dtype=int32)

In [26]:
chunks[0]['variants/AN']

array([-1, -1, -1], dtype=int32)

In [27]:
chunks[0]['variants/DB']

array([False, False,  True], dtype=bool)

In [28]:
chunks[0]['variants/H2']

array([False, False,  True], dtype=bool)

In [29]:
chunks[1]['variants/AA']

array([[b'', b''],
       [b'T', b''],
       [b'T', b'']], 
      dtype='|S12')

In [30]:
chunks[0]['calldata/GT']

array([[[0, 0],
        [0, 0],
        [0, 1]],

       [[0, 0],
        [0, 0],
        [0, 1]],

       [[0, 0],
        [1, 0],
        [1, 1]]], dtype=int8)

In [31]:
chunks[0]['calldata/GQ']

array([[-1, -1, -1],
       [-1, -1, -1],
       [48, 48, 43]], dtype=int8)

In [32]:
chunks[0]['calldata/HQ']

array([[[10, 10],
        [10, 10],
        [ 3,  3]],

       [[10, 10],
        [10, 10],
        [ 3,  3]],

       [[51, 51],
        [51, 51],
        [-1, -1]]], dtype=int8)

In [33]:
chunks[0]['calldata/DP']

array([[-1, -1, -1],
       [-1, -1, -1],
       [ 1,  8,  5]], dtype=int16)

In [34]:
callset = read_vcf(sample_vcf_fn, buffer_size=2**15, chunk_length=1000)
sorted(callset.keys())

['calldata/GT',
 'samples',
 'variants/ALT',
 'variants/CHROM',
 'variants/FILTER_PASS',
 'variants/ID',
 'variants/POS',
 'variants/QUAL',
 'variants/REF']

In [35]:
callset = read_vcf(sample_vcf_fn, fields='FILTER',
                   buffer_size=2**15, chunk_length=1000)

In [36]:
sorted(callset.keys())

['samples',
 'variants/FILTER_PASS',
 'variants/FILTER_q10',
 'variants/FILTER_s50']

In [37]:
callset['samples']

array([b'NA00001', b'NA00002', b'NA00003'], 
      dtype='|S7')

In [38]:
callset['variants/FILTER_q10']

array([False, False, False,  True, False, False, False, False, False], dtype=bool)

In [39]:
callset['variants/FILTER_s50']

array([False, False, False, False, False, False, False, False, False], dtype=bool)

In [40]:
callset = read_vcf(sample_vcf_fn, fields='variants/*')

In [41]:
callset = read_vcf(sample_vcf_fn, fields='*', numbers=dict(ALT=1, AA=1),
                   buffer_size=2**15, chunk_length=1000)
sorted(callset.keys())

['calldata/DP',
 'calldata/GQ',
 'calldata/GT',
 'calldata/HQ',
 'samples',
 'variants/AA',
 'variants/AC',
 'variants/AF',
 'variants/ALT',
 'variants/AN',
 'variants/CHROM',
 'variants/DB',
 'variants/DP',
 'variants/FILTER_PASS',
 'variants/FILTER_q10',
 'variants/FILTER_s50',
 'variants/H2',
 'variants/ID',
 'variants/NS',
 'variants/POS',
 'variants/QUAL',
 'variants/REF']

In [42]:
callset['variants/ALT'].shape

(9,)

In [43]:
callset['variants/ALT']

array([b'C', b'G', b'A', b'A', b'G', b'.', b'G', b'.', b'A'], 
      dtype='|S1')

In [44]:
callset['variants/AA']

array([b'', b'', b'', b'', b'T', b'T', b'G', b'', b''], 
      dtype='|S12')

In [45]:
callset = read_vcf(prof_vcf_fn, buffer_size=2**15, chunk_length=1000)

In [46]:
sorted(callset.keys())

['calldata/GT',
 'samples',
 'variants/ALT',
 'variants/CHROM',
 'variants/FILTER_PASS',
 'variants/ID',
 'variants/POS',
 'variants/QUAL',
 'variants/REF']

In [47]:
callset['samples']

array([b'AB0085-C', b'AB0087-C', b'AB0088-C', b'AB0089-C', b'AB0090-C',
       b'AB0091-C', b'AB0092-C', b'AB0094-C', b'AB0095-C', b'AB0097-C',
       b'AB0098-C', b'AB0099-C', b'AB0100-C', b'AB0101-C', b'AB0103-C',
       b'AB0104-C', b'AB0109-C', b'AB0110-C', b'AB0111-C', b'AB0112-C',
       b'AB0113-C', b'AB0114-C', b'AB0117-C', b'AB0119-C', b'AB0122-C',
       b'AB0123-C', b'AB0124-C', b'AB0126-C', b'AB0127-C', b'AB0128-C',
       b'AB0129-C', b'AB0130-C', b'AB0133-C', b'AB0134-C', b'AB0135-C',
       b'AB0136-C', b'AB0137-C', b'AB0138-C', b'AB0139-C', b'AB0140-C',
       b'AB0142-C', b'AB0143-C', b'AB0145-C', b'AB0146-C', b'AB0147-C',
       b'AB0148-C', b'AB0151-C', b'AB0153-C', b'AB0155-C', b'AB0157-C',
       b'AB0158-C', b'AB0159-C', b'AB0160-C', b'AB0161-C', b'AB0164-C',
       b'AB0166-C', b'AB0169-C', b'AB0170-C', b'AB0171-C', b'AB0172-C',
       b'AB0173-C', b'AB0174-C', b'AB0175-C', b'AB0176-C', b'AB0177-C',
       b'AB0178-C', b'AB0179-C', b'AB0181-C', b'AB0182-C', b'AB0

In [48]:
callset['calldata/GT'].shape

(1967, 773, 2)

In [49]:
callset['calldata/GT'].shape

(1967, 773, 2)

In [50]:
callset['variants/CHROM']

array([b'2L', b'2L', b'2L', ..., b'2L', b'2L', b'2L'], 
      dtype='|S12')

In [51]:
callset['variants/POS']

array([2353212, 2353223, 2353234, ..., 2436558, 2436585, 2436615], dtype=int32)

In [52]:
callset['variants/REF']

array([b'G', b'T', b'G', ..., b'G', b'A', b'C'], 
      dtype='|S1')

In [53]:
callset['variants/ALT']

array([[b'A', b'', b''],
       [b'G', b'', b''],
       [b'C', b'', b''],
       ..., 
       [b'A', b'', b''],
       [b'C', b'', b''],
       [b'A', b'', b'']], 
      dtype='|S1')

## Format conversion

In [54]:
npz_fn = 'sample.npz'
vcf_to_npz(sample_vcf_fn, npz_fn, fields='*', chunk_length=3, overwrite=False)

ValueError: file exists at path 'sample.npz'; use overwrite=True to replace

In [55]:
vcf_to_npz(sample_vcf_fn, npz_fn, fields='*', chunk_length=3, overwrite=True)

In [56]:
!ls -lh {npz_fn}

-rw-r--r-- 1 aliman aliman 4.3K May 30 10:38 sample.npz


In [57]:
import numpy as np

In [58]:
callset = np.load(npz_fn)
callset

In [59]:
sorted(callset.keys())

['calldata/DP',
 'calldata/GQ',
 'calldata/GT',
 'calldata/HQ',
 'samples',
 'variants/AA',
 'variants/AC',
 'variants/AF',
 'variants/ALT',
 'variants/AN',
 'variants/CHROM',
 'variants/DB',
 'variants/DP',
 'variants/FILTER_PASS',
 'variants/FILTER_q10',
 'variants/FILTER_s50',
 'variants/H2',
 'variants/ID',
 'variants/NS',
 'variants/POS',
 'variants/QUAL',
 'variants/REF']

In [60]:
callset['samples']

array([b'NA00001', b'NA00002', b'NA00003'], 
      dtype='|S7')

In [61]:
callset['variants/POS']

array([    111,     112,   14370,   17330, 1110696, 1230237, 1234567,
       1235237,      10], dtype=int32)

In [62]:
callset['variants/CHROM']

array([b'19', b'19', b'20', b'20', b'20', b'20', b'20', b'20', b'X'], 
      dtype='|S12')

In [63]:
callset['calldata/GT']

array([[[ 0,  0],
        [ 0,  0],
        [ 0,  1]],

       [[ 0,  0],
        [ 0,  0],
        [ 0,  1]],

       [[ 0,  0],
        [ 1,  0],
        [ 1,  1]],

       [[ 0,  0],
        [ 0,  1],
        [ 0,  0]],

       [[ 1,  2],
        [ 2,  1],
        [ 2,  2]],

       [[ 0,  0],
        [ 0,  0],
        [ 0,  0]],

       [[ 0,  1],
        [ 0,  2],
        [-1, -1]],

       [[ 0,  0],
        [ 0,  0],
        [-1, -1]],

       [[ 0, -1],
        [ 0,  1],
        [ 0,  2]]], dtype=int8)

In [64]:
hdf5_fn = 'sample.h5'
vcf_to_hdf5(sample_vcf_fn, hdf5_fn, fields='*', chunk_length=3)

ValueError: dataset exists at path 'samples'; use overwrite=True to replace

In [65]:
vcf_to_hdf5(sample_vcf_fn, hdf5_fn, fields='*', chunk_length=3, overwrite=True)

In [66]:
!ls -lh {hdf5_fn}

-rw-r--r-- 1 aliman aliman 82K May 30 10:38 sample.h5


In [67]:
!h5ls {hdf5_fn}

calldata                 Group
samples                  Dataset {3}
variants                 Group


In [68]:
!h5ls {hdf5_fn}/variants

AA                       Dataset {9/Inf}
AC                       Dataset {9/Inf, 3}
AF                       Dataset {9/Inf, 3}
ALT                      Dataset {9/Inf, 3}
AN                       Dataset {9/Inf}
CHROM                    Dataset {9/Inf}
DB                       Dataset {9/Inf}
DP                       Dataset {9/Inf}
FILTER                   Dataset {9/Inf, 1}
FILTER_PASS              Dataset {9/Inf}
FILTER_q10               Dataset {9/Inf}
FILTER_s50               Dataset {9/Inf}
H2                       Dataset {9/Inf}
ID                       Dataset {9/Inf}
NS                       Dataset {9/Inf}
POS                      Dataset {9/Inf}
QUAL                     Dataset {9/Inf}
REF                      Dataset {9/Inf}


In [69]:
!h5ls {hdf5_fn}/variants/CHROM

CHROM                    Dataset {9/Inf}


In [70]:
!h5ls {hdf5_fn}/calldata

DP                       Dataset {9/Inf, 3}
GQ                       Dataset {9/Inf, 3}
GT                       Dataset {9/Inf, 3, 2}
HQ                       Dataset {9/Inf, 3, 2}


In [71]:
!h5ls {hdf5_fn}/calldata/GT

GT                       Dataset {9/Inf, 3, 2}


In [72]:
import h5py

In [73]:
with h5py.File(hdf5_fn, mode='r') as h5f:
    print(h5f['samples'][:])
    print(h5f['variants/CHROM'][:])
    print(h5f['variants/POS'][:])
    print(h5f['calldata/GT'][:])
    

[b'NA00001' b'NA00002' b'NA00003']
[b'19' b'19' b'20' b'20' b'20' b'20' b'20' b'20' b'X']
[    111     112   14370   17330 1110696 1230237 1234567 1235237      10]
[[[ 0  0]
  [ 0  0]
  [ 0  1]]

 [[ 0  0]
  [ 0  0]
  [ 0  1]]

 [[ 0  0]
  [ 1  0]
  [ 1  1]]

 [[ 0  0]
  [ 0  1]
  [ 0  0]]

 [[ 1  2]
  [ 2  1]
  [ 2  2]]

 [[ 0  0]
  [ 0  0]
  [ 0  0]]

 [[ 0  1]
  [ 0  2]
  [-1 -1]]

 [[ 0  0]
  [ 0  0]
  [-1 -1]]

 [[ 0 -1]
  [ 0  1]
  [ 0  2]]]


In [74]:
zarr_fn = 'sample.zarr'
vcf_to_zarr(sample_vcf_fn, zarr_fn, fields='*', chunk_length=3)

KeyError: "path 'samples' contains an array"

In [75]:
vcf_to_zarr(sample_vcf_fn, zarr_fn, fields='*', chunk_length=3, overwrite=True)

In [76]:
import zarr
callset = zarr.open_group('sample.zarr')
callset

Group(/, 3)
  arrays: 1; samples
  groups: 2; calldata, variants
  store: DirectoryStore

In [77]:
callset['samples'][:]

array([b'NA00001', b'NA00002', b'NA00003'], 
      dtype='|S7')

In [78]:
callset['variants']

Group(/variants, 18)
  arrays: 18; AA, AC, AF, ALT, AN, CHROM, DB, DP, FILTER, FILTER_PASS, FILTE...
  store: DirectoryStore

In [79]:
callset['variants/POS'][:]

array([    111,     112,   14370,   17330, 1110696, 1230237, 1234567,
       1235237,      10], dtype=int32)

In [80]:
callset['variants/CHROM'][:]

array([b'19', b'19', b'20', b'20', b'20', b'20', b'20', b'20', b'X'], 
      dtype='|S12')

In [81]:
callset['calldata']

Group(/calldata, 4)
  arrays: 4; DP, GQ, GT, HQ
  store: DirectoryStore

In [82]:
callset['calldata/GT'][:]

array([[[ 0,  0],
        [ 0,  0],
        [ 0,  1]],

       [[ 0,  0],
        [ 0,  0],
        [ 0,  1]],

       [[ 0,  0],
        [ 1,  0],
        [ 1,  1]],

       [[ 0,  0],
        [ 0,  1],
        [ 0,  0]],

       [[ 1,  2],
        [ 2,  1],
        [ 2,  2]],

       [[ 0,  0],
        [ 0,  0],
        [ 0,  0]],

       [[ 0,  1],
        [ 0,  2],
        [-1, -1]],

       [[ 0,  0],
        [ 0,  0],
        [-1, -1]],

       [[ 0, -1],
        [ 0,  1],
        [ 0,  2]]], dtype=int8)

In [83]:
callset['calldata/GQ'][:]

array([[-1, -1, -1],
       [-1, -1, -1],
       [48, 48, 43],
       [49,  3, 41],
       [21,  2, 35],
       [54, 48, 61],
       [-1, 17, 40],
       [-1, -1, -1],
       [-1, -1, -1]], dtype=int8)

## Profiling

In [9]:
%time _ = read_vcf(prof_vcf_fn, buffer_size=2**15, chunk_length=1000)

CPU times: user 132 ms, sys: 4 ms, total: 136 ms
Wall time: 135 ms


In [10]:
%timeit _ = read_vcf(prof_vcf_fn, buffer_size=2**15, chunk_length=1000)

10 loops, best of 3: 91.6 ms per loop


In [11]:
import cProfile

In [12]:
cProfile.run('read_vcf(prof_vcf_fn, buffer_size=2**15, chunk_length=1000)', sort='time')

         46648 function calls (46264 primitive calls) in 0.150 seconds

   Ordered by: internal time

   ncalls  tottime  percall  cumtime  percall filename:lineno(function)
        1    0.077    0.077    0.117    0.117 {allel.opt.io_vcf_read.iter_vcf}
      377    0.022    0.000    0.022    0.000 {method 'decompress' of 'zlib.Decompress' objects}
      473    0.010    0.000    0.010    0.000 {built-in method zlib.crc32}
     7744    0.009    0.000    0.063    0.000 gzip.py:269(read)
8128/7744    0.009    0.000    0.048    0.000 {method 'read' of '_io.BufferedReader' objects}
       12    0.006    0.001    0.030    0.002 io_vcf_read.py:745(_binary_readline)
     7744    0.004    0.000    0.006    0.000 _compression.py:12(_check_not_closed)
     7746    0.002    0.000    0.002    0.000 gzip.py:296(closed)
      378    0.002    0.000    0.038    0.000 gzip.py:436(read)
      378    0.002    0.000    0.040    0.000 _compression.py:66(readinto)
      949    0.002    0.000    0.002    0.000

In [95]:
%time vcf_to_npz(prof_vcf_fn, 'prof.npz', buffer_size=2**15, chunk_length=1000, overwrite=True)

CPU times: user 224 ms, sys: 4 ms, total: 228 ms
Wall time: 230 ms


In [96]:
%time vcf_to_hdf5(prof_vcf_fn, 'prof.h5', buffer_size=2**15, chunk_length=1000, overwrite=True)

CPU times: user 180 ms, sys: 0 ns, total: 180 ms
Wall time: 179 ms


In [97]:
%time vcf_to_zarr(prof_vcf_fn, 'prof.zarr', buffer_size=2**15, chunk_length=1000, overwrite=True)

CPU times: user 168 ms, sys: 16 ms, total: 184 ms
Wall time: 176 ms


In [66]:
%prun vcf_to_hdf5(prof_vcf_fn, 'prof.h5', buffer_size=2**15, chunk_length=1000, overwrite=True)

In [67]:
%prun vcf_to_zarr(prof_vcf_fn, 'prof.zarr', buffer_size=2**15, chunk_length=1000, overwrite=True)

In [11]:
import line_profiler
l = line_profiler.LineProfiler()
# l.add_function(_read_vcf)
l.add_function(iter_vcf)
# l.add_function(CalldataParser_parse)
l.add_function(GenotypeInt8Parser_parse)
# l.add_function(ParserContext_next)
# l.add_function(BufferedReader_read)
l.runcall(read_vcf, prof_vcf_fn, buffer_size=2**15, chunk_length=1000)
l.print_stats()

Timer unit: 1e-06 s

Total time: 23.9421 s
File: /home/aliman/src/github/cggh/scikit-allel/allel/opt/io_vcf_read.pyx
Function: iter_vcf at line 71

Line #      Hits         Time  Per Hit   % Time  Line Contents
    71                                           def iter_vcf(binary_file, int buffer_size, int chunk_length, int temp_max_size, headers, fields,
    72                                                        types, numbers):
    73                                               cdef:
    74                                                   ParserContext context
    75                                                   Parser chrom_parser
    76                                                   Parser pos_parser
    77                                                   Parser id_parser
    78                                                   Parser ref_parser
    79                                                   Parser alt_parser
    80                                            

In [26]:
import vcfnp

In [28]:
%time vcfnp.calldata(prof_vcf_fn, fields=('genotype',))

[vcfnp] 2017-05-24 14:49:30.651145 :: caching is disabled
[vcfnp] 2017-05-24 14:49:30.651788 :: building array


CPU times: user 4.31 s, sys: 0 ns, total: 4.31 s
Wall time: 4.3 s


array([ (([0, 0],), ([0, 0],), ([0, 0],), ([0, 0],), ([0, 0],), ([0, 0],), ([0, 0],), ([0, 0],), ([0, 0],), ([0, 0],), ([0, 0],), ([0, 0],), ([0, 0],), ([0, 0],), ([0, 0],), ([0, 0],), ([0, 0],), ([0, 0],), ([0, 0],), ([0, 0],), ([0, 0],), ([0, 0],), ([0, 0],), ([0, 0],), ([0, 0],), ([0, 0],), ([0, 0],), ([0, 0],), ([0, 0],), ([0, 0],), ([0, 0],), ([0, 0],), ([0, 0],), ([0, 0],), ([0, 0],), ([0, 0],), ([0, 0],), ([0, 0],), ([0, 0],), ([0, 0],), ([0, 0],), ([0, 0],), ([0, 0],), ([0, 0],), ([0, 0],), ([0, 0],), ([0, 0],), ([0, 0],), ([0, 0],), ([0, 0],), ([0, 0],), ([0, 0],), ([0, 0],), ([0, 0],), ([0, 0],), ([0, 0],), ([0, 0],), ([0, 0],), ([0, 0],), ([0, 0],), ([0, 0],), ([0, 0],), ([0, 0],), ([0, 0],), ([0, 0],), ([0, 0],), ([0, 0],), ([0, 0],), ([0, 0],), ([0, 0],), ([0, 0],), ([0, 0],), ([0, 0],), ([0, 0],), ([0, 0],), ([0, 0],), ([0, 0],), ([0, 0],), ([0, 0],), ([0, 0],), ([0, 0],), ([0, 0],), ([0, 0],), ([0, 0],), ([0, 0],), ([0, 0],), ([0, 0],), ([0, 0],), ([0, 0],), ([0, 0],), (

In [29]:
4.3 / 0.09

47.77777777777778

## Legacy

In [ ]:
vcf_block_read(vcf_fn, buffer_size=2**15, block_size=2**25)

In [3]:
%time spike_read_len(vcf_fn, buffer_size=10)

CPU times: user 700 ms, sys: 0 ns, total: 700 ms
Wall time: 697 ms


6140661

In [4]:
%timeit spike_read_len(vcf_fn, buffer_size=100)

10 loops, best of 3: 105 ms per loop


In [5]:
%timeit spike_read_len(vcf_fn, buffer_size=1000)

10 loops, best of 3: 50 ms per loop


In [6]:
%timeit spike_read_len(vcf_fn, buffer_size=2**15)

10 loops, best of 3: 39 ms per loop


In [7]:
%timeit spike_read_len(vcf_fn, buffer_size=2**12)

10 loops, best of 3: 45.3 ms per loop


In [8]:
import cProfile

In [9]:
cProfile.run('spike_read_len(vcf_fn, buffer_size=2**15)', sort='time')

         6146762 function calls (6146566 primitive calls) in 0.941 seconds

   Ordered by: internal time

   ncalls  tottime  percall  cumtime  percall filename:lineno(function)
        1    0.584    0.584    0.941    0.941 io_vcf.pyx:90(spike_read_len)
  6140662    0.322    0.000    0.356    0.000 io_vcf.pyx:74(BufferedInputStream_next)
      189    0.019    0.000    0.019    0.000 {method 'decompress' of 'zlib.Decompress' objects}
      285    0.008    0.000    0.008    0.000 {built-in method zlib.crc32}
  385/189    0.001    0.000    0.033    0.000 {method 'read' of '_io.BufferedReader' objects}
      190    0.001    0.000    0.031    0.000 gzip.py:436(read)
      190    0.001    0.000    0.033    0.000 _compression.py:66(readinto)
      761    0.001    0.000    0.001    0.000 gzip.py:80(read)
      189    0.001    0.000    0.034    0.000 gzip.py:269(read)
      189    0.000    0.000    0.035    0.000 io_vcf.pyx:56(BufferedInputStream_fill_buffer)
       97    0.000    0.000    0.00

In [10]:
import line_profiler

l = line_profiler.LineProfiler()
l.add_function(spike_read_len)
l.add_function(BufferedInputStream_next)
l.add_function(BufferedInputStream_fill_buffer)
l.runcall(spike_read_len, vcf_fn, buffer_size=2**14)
l.print_stats()

## Legacy

In [ ]:
l = line_profiler.CLineProfiler

In [3]:
2**15

32768

In [ ]:
blocks = io_vcf.vcf_block_read(vcf_fn, buffer_size=2**16, block_size=1000)

HeaderParser_parse 37873392 35
20
b'##fileformat=VCFv4.1'
HeaderParser_parse 37873413 35
60
b'##FORMAT=<ID=GT,Number=1,Type=String,Description="Genotype">'
HeaderParser_parse 37873474 35
124
b'##INFO=<ID=AC,Number=A,Type=Integer,Description="Allele countin genotypes, for each ALT allele, in the same order aslisted">'
HeaderParser_parse 37873599 35
32
b'##contig=<ID=2L,length=49364325>'
HeaderParser_parse 37873632 35
32
b'##contig=<ID=2R,length=61545105>'
HeaderParser_parse 37873665 35
32
b'##contig=<ID=3L,length=41963435>'
HeaderParser_parse 37873698 35
32
b'##contig=<ID=3R,length=53200684>'
HeaderParser_parse 37873731 35
34
b'##contig=<ID=UNKN,length=42389979>'
HeaderParser_parse 37873766 35
31
b'##contig=<ID=X,length=24393108>'
HeaderParser_parse 37873798 35
38
b'##contig=<ID=Y_unplaced,length=237045>'
HeaderParser_parse 37873837 35
106
b'##reference=file:///data/anopheles/ag1000g/data/genome/AgamP3/Anopheles-gambiae-PEST_CHROMOSOMES_AgamP3.fa'
HeaderParser_parse 37873944 35
7002
